In [1]:
import pandas as pd
import tensorflow as tf

df = pd.read_csv('/content/sample_data/profanity.csv')
df = df[['sentence', 'label']]
df = df.dropna()

df.head()

,sentence,label
0,Son of a bitch took my Tic Tacs,1.0
1,Young Pill Chamberlain these bitches love my m...,1.0
2,Fuck Red Malone man bitch ass niggah,1.0
3,Why the eggplant emoji doe?,1.0
4,0 rings 0 mvps 0 bitches lol,1.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [4]:
from sklearn.model_selection import train_test_split
import numpy as np
x = df['sentence'].to_list()
y = df['label'].to_list()

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2, random_state=42)


In [5]:
import tensorflow_hub as hub
from tensorflow.keras.layers import TextVectorization
max_vocab = 10000
max_length = 20

text_vectorizer = TextVectorization(max_tokens=max_vocab,
                                    output_mode="int",
                                    output_sequence_length=max_length)
text_vectorizer.adapt(x_train)

In [6]:
class CustomTextVectorization(tf.keras.layers.Layer):
    def __init__(self, max_tokens, output_sequence_length, **kwargs):
        super(CustomTextVectorization, self).__init__(**kwargs)
        self.max_tokens = max_tokens
        self.output_sequence_length = output_sequence_length
        self.text_vectorizer = tf.keras.layers.TextVectorization(
            max_tokens=max_tokens,
            output_mode="int",
            output_sequence_length=output_sequence_length,
        )

    def call(self, inputs):
        return self.text_vectorizer(inputs)

custom_text_vectorizer = CustomTextVectorization(max_tokens=max_vocab, output_sequence_length=max_length)

In [7]:
from tensorflow.keras import layers
Embedder = layers.Embedding(input_dim=max_vocab,
                                     output_dim=20,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding")


In [8]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = Embedder(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs, name="Model")

In [9]:
model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


model_history = model.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val))

Epoch 1/100
5/5 [==============================] - 6s 267ms/step - loss: 0.6927 - accuracy: 0.5141 - val_loss: 0.6991 - val_accuracy: 0.3333
Epoch 2/100
5/5 [==============================] - 0s 35ms/step - loss: 0.6874 - accuracy: 0.5423 - val_loss: 0.6903 - val_accuracy: 0.5556
Epoch 3/100
5/5 [==============================] - 0s 52ms/step - loss: 0.6795 - accuracy: 0.6338 - val_loss: 0.6839 - val_accuracy: 0.6667
Epoch 4/100
5/5 [==============================] - 0s 84ms/step - loss: 0.6642 - accuracy: 0.8099 - val_loss: 0.6660 - val_accuracy: 0.8889
Epoch 5/100
5/5 [==============================] - 0s 75ms/step - loss: 0.6299 - accuracy: 0.9155 - val_loss: 0.6007 - val_accuracy: 0.8333
Epoch 6/100
5/5 [==============================] - 0s 75ms/step - loss: 0.5546 - accuracy: 0.8873 - val_loss: 0.4924 - val_accuracy: 0.8333
Epoch 7/100
5/5 [==============================] - 0s 67ms/step - loss: 0.3964 - accuracy: 0.9225 - val_loss: 0.3486 - val_accuracy: 0.8889
Epoch 8/100
5/5 [==

In [10]:
def final_pred(quote):
  val = model.predict([quote])
  # Round
  rounded = np.round(val)

  # Convert to integer
  a_int = rounded.astype(int)
  return a_int[0][0]

print(final_pred(['I love your mom']))

1/1 [==============================] - 1s 1s/step
0


In [11]:
model.save('hate_speech.tf')